In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hsankesara/medium-articles")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/medium-articles


In [27]:
import pandas as pd
import numpy as np
from transformers import BartTokenizer, BartForConditionalGeneration

In [4]:
df = pd.read_csv('/kaggle/input/medium-articles/articles.csv')
df.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


In [5]:
df.shape

(337, 6)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author        337 non-null    object
 1   claps         337 non-null    object
 2   reading_time  337 non-null    int64 
 3   link          337 non-null    object
 4   title         337 non-null    object
 5   text          337 non-null    object
dtypes: int64(1), object(5)
memory usage: 15.9+ KB


In [8]:
df.apply(pd.isnull).sum()

author          0
claps           0
reading_time    0
link            0
title           0
text            0
dtype: int64

In [9]:
df.duplicated().sum()

0

In [10]:
df['text'][0]

"Oh, how the headlines blared:\nChatbots were The Next Big Thing.\nOur hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start socializing with machines.\nAnd why wouldn’t they be? All the road signs pointed towards insane success.\nAt the Mobile World Congress 2017, chatbots were the main headliners. The conference organizers cited an ‘overwhelming acceptance at the event of the inevitable shift of focus for brands and corporates to chatbots’.\nIn fact, the only significant question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place:\nOne year on, we have an answer to that question.\nNo.\nBecause there isn’t even an ecosystem for a platform to dominate.\nChatbots weren’t the first technological development to be talked up in grandiose terms and then slump spectacularly.\nThe age-old hype cycle unfolded in familiar fashion...\nExpectations built, built, and then...

In [28]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [29]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [30]:
def abstractive_sum(text):
    tokenized_text = tokenizer(text, max_length = 1024, truncation = True, return_tensors = 'pt')
    summ = model.generate(tokenized_text['input_ids'], min_length = 500, max_length = 1500, length_penalty = 2.0, num_beams = 4,
                         early_stopping = True)
    out = tokenizer.decode(summ[0], skip_special_tokens = True)
    return out

In [31]:
abstractive_sum(df['text'][0])

"Chatbots were the main headliners at the Mobile World Congress 2017. The only question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place. There isn’t even an ecosystem for a platform to dominate. The big divide is text vs. speech. A great bot can be about as useful as an average app. When it comes to competition, there’s no ‘extend’ ‘replace’ word with ‘chatbot’. The age-old hype cycle unfolded in familiar fashion...Expectations built, built, and then..... It all kind of fizzled out. The predicted paradim shift didn't materialize. And apps are, tellingly, still alive and well. We’re still trying to achieve the same innovations we were 30 years ago. Are chatbots cheaper or faster than apps? No — not yet, at least. Do you remember when apps first came onto the scene ten years ago: but do you Remember when apps replaced the internet? The answer is yes, of course you do, but you’ll have to wait a few years to find out if they’ve mad

In [36]:
def headline(text):
    tokenized_text = tokenizer(text, max_length = 1024, return_tensors = 'pt', truncation = True)
    summ = model.generate(tokenized_text['input_ids'], min_length = 3, max_length = 10, length_penalty = 2.0, num_beams = 4,
                  early_stopping = True)
    out = tokenizer.decode(summ[0], skip_special_tokens = True)
    return out

In [37]:
headline(df['text'][0])

'Chatbots were the main headliners'